# 数据预处理

在数据集上训练模型之前，数据需要被预处理为模型期望的输入格式。无论数据是文本、图像还是音频，它们都需要被转换并组合成批量的张量。

🤗 Transformers 提供了一组预处理类来帮助准备数据以供模型使用。在本教程中，你将了解以下内容：
1. 对于文本，使用[分词器(Tokenizer)](https://huggingface.co/docs/transformers/v4.44.2/zh/main_classes/tokenizer)能够将文本转换为一系列标记`tokens`，并创建 tokens 的数字表示，将它们组合成张量。
2. 对于语音和音频，使用[特征提取器(Feature extractor)](https://huggingface.co/docs/transformers/v4.44.2/zh/main_classes/feature_extractor)能够从音频波形中提取顺序特征并将其转换为张量。
3. 对于图像，使用`图像处理器(ImageProcessor)`将图像转换为张量。
4. 对于多模态输入，使用[处理器(Processor)](https://huggingface.co/docs/transformers/v4.44.2/zh/main_classes/processors)，其结合了 Tokenizer 和 ImageProcessor 或 Processor。

`AutoProcessor` 能够有效的自动选择适用于模型输入的预处理工具，无论你使用的是 `Tokenizer`、`ImageProcessor`、`Feature extractor`还是`Processor`。

在开始之前，请安装`🤗 Datasets`，可以加载一些数据集来进行实验：

In [ ]:
pip install datasets

## 自然语言处理